# Prostate cANcer graDe Assessment (PANDA) Challenge
### Prostate cancer diagnosis using the Gleason grading system

In [ ]:
import sys
sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path

In [ ]:
# !pip install torchsummary

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.callbacks import *
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score,confusion_matrix
import matplotlib.image as image
from tqdm.notebook import tqdm
import os
import gc
import zipfile
import openslide
import cv2
from PIL import Image
import skimage.io as sk
import warnings
# from torchsummary import summary
from sys import getsizeof
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device('cuda')

In [ ]:
tile_size = 256
image_size = 256
n_tiles = 36
batch_size = 8
num_workers = 4
TRAIN = '/kaggle/input/prostate-cancer-grade-assessment/train_images/'

In [ ]:
path1 = Path('/kaggle/input/panda-36-tiles')

In [ ]:
sld = os.listdir(TRAIN)
sld = [x[:-5] for x in sld]

In [ ]:
df_duplicates = pd.read_csv('../input/duplicates-panda/duplicates.csv')
# df_duplicates.head()

In [ ]:
duplicate_files = df_duplicates['file2'].tolist()

In [ ]:
df = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/train.csv')
df = df[df['image_id'].isin(sld)]
df = df[~df['image_id'].isin(duplicate_files)]
df.columns = ['fn', 'data_provider', 'isup_grade', 'gleason_score']

## Stratified Kfold

In [ ]:
df['kfold'] = -1
df = df.sample(frac=1.,random_state=2020).reset_index(drop=True)
kf = StratifiedKFold(n_splits=5)
y = df.isup_grade.values
for f,(t_,v_) in enumerate(kf.split(X=df,y=y)):
    df.loc[v_,'kfold'] = f

In [ ]:
# df.head()

In [ ]:
import seaborn as sns
sns.countplot(x=df[df.kfold==1].isup_grade);
plt.title('Fold - 1: Images count');

# Data Processing for fastai 
* We have 2 options either we write a custom Imagelist function or
* We first convert all images first then use then As we like.

Later will take time at first but will Speed up process later. As Fastai datablock will not have to process large **.tiff** files every time

* I have converted the tiff files they can be found [**here**](https://www.kaggle.com/ianmoone0617/panda-36-tiles-resize)
* Lets start with Custum ImageItem List first

In [ ]:
def get_tiles(img, mode=0):
        result = []
        h, w, c = img.shape
        pad_h = (tile_size - h % tile_size) % tile_size + ((tile_size * mode) // 2)
        pad_w = (tile_size - w % tile_size) % tile_size + ((tile_size * mode) // 2)

        img2 = np.pad(img,[[pad_h // 2, pad_h - pad_h // 2], [pad_w // 2,pad_w - pad_w//2], [0,0]], constant_values=255)
        img3 = img2.reshape(
            img2.shape[0] // tile_size,
            tile_size,
            img2.shape[1] // tile_size,
            tile_size,
            3
        )

        img3 = img3.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size,3)
        n_tiles_with_info = (img3.reshape(img3.shape[0],-1).sum(1) < tile_size ** 2 * 3 * 255).sum()
        if len(img) < n_tiles:
            img3 = np.pad(img3,[[0,N-len(img3)],[0,0],[0,0],[0,0]], constant_values=255)
        idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:n_tiles]
        img3 = img3[idxs]
        for i in range(len(img3)):
            result.append({'img':img3[i], 'idx':i})
        return result

## Custom Fastai TiffImageList to Directly Process Slides

In [ ]:
class TiffImageItemList(ImageList):
    def open(self,fn):
        path = '/kaggle/input/prostate-cancer-grade-assessment/train_images/'
        fl = path + str(fn)+'.tiff'
        img = sk.MultiImage(fl)[1]
        res = get_tiles(img)
        imgs = []
        for i in range(36):
            im = res[i%len(res)]['img']
            imgs.append(im)
        imgs = np.array(imgs)
        final_image = np.concatenate(np.array([np.concatenate(imgs[j:j+6],axis=1).astype(np.uint8) for j in range(0,36,6)]),axis=0)
        final_image = cv2.resize(final_image, dsize=(512, 512), interpolation=cv2.INTER_CUBIC)
        
        return vision.Image(pil2tensor(final_image,np.float32).div_(255))

* Train and validation split

In [ ]:
trn_idx,val_idx = list(df[df.kfold!=4].index),list(df[df.kfold==4].index)
random.shuffle(trn_idx)
random.shuffle(val_idx)

## DataBunch of Custom TiffImageItemList 

In [ ]:
data = (TiffImageItemList.from_df(df,path='',cols='fn')
                          .split_by_idxs(trn_idx,val_idx)
                          .label_from_df(cols='isup_grade')
                          .transform(get_transforms())
                          .databunch(num_workers=4,bs=batch_size)
                          .normalize(imagenet_stats))


In [ ]:
#data.show_batch(rows=3,figsize=(20,8))

## Databunch of Processed Images: Using fastai's own ImageList

In [ ]:
# stats = ([0.785946], [0.45007266])
# data_img = (ImageList.from_df(df,path1,folder='.',suffix='.png',cols='fn')
#                 .split_by_idxs(trn_idx,val_idx)
#                 .label_from_df(cols='isup_grade',)
#                 .transform(get_transforms(do_flip=True), size=300)
#                 .databunch(bs=batch_size).normalize(imagenet_stats))

In [ ]:
# data_img.show_batch(rows=3,figsize=(20,8),seed=2020)

In [ ]:
data_img = data

In [ ]:
len(data_img.train_ds), len(data_img.valid_ds), data_img.classes, data_img.train_ds[0][0].data.shape,data_img.c

## Model Efficient-B3

In [ ]:
from efficientnet_pytorch import model as enet

In [ ]:
pretrained_model = {
    'efficientnet-b3': '../input/efficientnet-pytorch/efficientnet-b3-c8376fa2.pth'
}

enet_type = 'efficientnet-b3'
out_dim = 6

In [ ]:
class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        super(enetv2, self).__init__()
        self.enet = enet.EfficientNet.from_name(backbone)
        self.enet.load_state_dict(torch.load(pretrained_model[backbone]))

        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()

    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.extract(x)
        x = self.myfc(x)
        return x

In [ ]:
arch = enetv2(enet_type, out_dim=out_dim)

### Metrics Kappa Score

In [ ]:
kp = KappaScore()
kp.weights = 'quadratic'

In [ ]:
learn = Learner(data_img, arch , metrics = [kp] , model_dir = '/kaggle/working/')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
gc.collect()

In [ ]:
cb2 = SaveModelCallback(learn, monitor = 'kappa_score', every = 'improvement', mode='max', name = 'best_model_ft' )
cb3 = ReduceLROnPlateauCallback(learn,  monitor = 'kappa_score', mode = 'max',factor = 0.2,patience = 4, min_delta = 0.01)

In [ ]:
learn.split([arch.myfc])

In [ ]:
epochs = 4
learn.fit_one_cycle(epochs ,max_lr = 1e-3, callbacks = [cb2,cb3])

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4 ,max_lr = 1e-3, callbacks = [cb2,cb3])

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.load('best_model_ft');

In [ ]:
learn.export('/kaggle/working/panda.pkl')

## Inference Kernel can be found [**here**](https://www.kaggle.com/ianmoone0617/panda-effnet-b3-inference-fastai-custom-imagelist)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
# learn = learn.to_fp32()

In [ ]:
# test_df = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/test.csv')
# df.drop('kfold', axis=1, inplace=True)
# df.columns = ['image_id', 'data_provider', 'isup_grade', 'gleason_score']
# data_dir = '../input/prostate-cancer-grade-assessment'
# image_folder = os.path.join(data_dir, 'test_images')
# is_test = os.path.exists(image_folder)  # IF test_images is not exists, we will use some train images.
# image_folder = image_folder if is_test else os.path.join(data_dir, 'train_images')

# test = test_df if is_test else df.sample(n=100)

In [ ]:
# def image_test(fn,image_folder):     
#     path2 = image_folder +'/'
#     fl = path2 + str(fn)+'.tiff'
#     img = sk.MultiImage(fl)[1]
#     res = get_tiles(img)
#     imgs = []
#     for i in range(36):
#         im = res[i%len(res)]['img']
#         imgs.append(im)
#     imgs = np.array(imgs)
#     final_image = np.concatenate(np.array([np.concatenate(imgs[j:j+6],axis=1).astype(np.uint8) for j in range(0,36,6)]),axis=0)
#     final_image = cv2.resize(final_image, dsize=(300, 300), interpolation=cv2.INTER_CUBIC)
#     return vision.Image(pil2tensor(final_image,np.float32).div_(255))

In [ ]:
# ts_name = test.image_id.values
# pred = np.zeros(len(ts_name))
    
# for j in tqdm(range(len(ts_name))):
#     ans = int(learn.predict(image_test(ts_name[j],image_folder))[0])
#     pred[j] = ans
        
# out = pd.DataFrame({'image_id':ts_name,'isup_grade':pred.astype(int)})
# out.to_csv('submission.csv',index=False)